##### 版权所有 2024 Google LLC.

In [ ]:
# @title 根据 Apache 许可证 2.0 版本（“许可证”）授权；
# 除非遵循许可证，否则不得使用本文件。
# 您可以通过以下地址获取许可证副本：
# https://www.apache.org/licenses/LICENSE-2.0
# 除非适用法律要求或书面同意，
# 否则根据“现状”分发本软件，不附带任何明示或暗示的保证。
# 详见许可证中关于权限与限制的具体条款。

# 在 SQL Spider 数据集上微调 CodeGemma
**作者**：Carlo Fisicaro  
**GitHub**：[github.com/carlofisicaro](https://github.com/carlofisicaro)  
**X**：[@carlo_fisicaro](https://twitter.com/carlo_fisicaro)

# CodeGemma text-to-sql（Hugging Face 版）
本笔记本演示如何借助 Hugging Face 加载、微调并部署 CodeGemma 模型，使其专注生成 SQL。
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/CodeGemma/[CodeGemma_1]Finetune_with_SQL.ipynb">
      <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 运行
    </a>
  </td>
</table>

## 环境准备

### 选择 Colab 运行时
要完成本教程，您需要具备足够资源运行 Gemma 模型的 Colab 运行时。此处推荐使用 T4 GPU：

1. 在 Colab 窗口右上角，点击 **▾（其他连接选项）**。  
2. 选择 **更改运行时类型**。  
3. 在 **硬件加速器** 下，选择 **T4 GPU**。  

### CodeGemma 初始化设置

**开始之前，请按以下步骤完成 CodeGemma 环境配置：**

1. **Hugging Face 账号**：如果尚未注册，可[点击此处](https://huggingface.co/join)免费创建。  
2. **申请模型权限**：前往 [CodeGemma 模型页面](google/codegemma-7b-it) 并接受使用条款。  
3. **选择高资源运行时**：在 Colab 中选择能够运行 Gemma 7B 模型的运行时。  
4. **生成 HF Token**：在[此处](https://huggingface.co/settings/tokens)新建一个具备 **write** 权限的 Access Token，后续步骤会用到。  

**完成以上步骤后，即可继续下一节，在 Colab 中配置环境变量。**

### 配置你的 HF Token
将 Hugging Face Token 添加到 Colab Secrets 管理器，以便安全存储。

1. 打开 Google Colab 笔记本，点击左侧面板中的 🔑 **Secrets** 标签。  
   <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="左侧面板中的 Secrets 标签" width=50%>
2. 新建一条 Secret，名称为 `HF_TOKEN`。  
3. 将您的 Token 复制到 `HF_TOKEN` 的 Value 输入框。  
4. 点击左侧开关，允许笔记本访问该 Secret。

### 安装依赖运行下方单元格，安装所有必需的依赖包。

### 登录 Hugging Face Hub

一切就绪，准备探索 Gemma 的无限可能！

## 实例化 CodeGemma 7B 模型

CodeGemma 是一系列强大而轻量的模型，能够完成多种编程任务：中间填充、代码生成、自然语言理解、数学推理及指令跟随等。  
此处我们加载 **7B 指令调优版**，用于自然语言到代码的对话与指令场景。

首先从 Hugging Face Hub 加载模型。

### 从 HF Hub 加载模型

我们先定义一段前置提示，让模型知道我们需要的是 SQL 查询。

### 小试牛刀

让我们向 CodeGemma 提一个**语义模糊**的问题

该问题之所以模糊，是因为不清楚我们到底想要：
- 一段**生成 SQL 的 Python 脚本**；
- 还是**分别生成 Python 与 SQL 的两段代码**。

CodeGemma 选择了第一种。请记住这一点！

## 使用 LoRA 微调模型
本节将聚焦于如何通过微调，让大模型根据自然语言生成高质量 SQL。

我们需要定义一个分词函数，对数据集的 `question` 和 `query` 两列进行 tokenize。

让我们用**同一个模糊问题**考考经过 SQL 微调的 CodeGemma

这一次，模型给出了**第二种答案**：分别提供 Python 与 SQL 两段代码！  
模型知道我们现在“偏爱”SQL，但它也没忘记自己曾训练过的其他语言。

## 将模型推送至你的 Hugging Face Hub

Hugging Face 支持一键上传，方便你保存与分享训练好的模型。

## 使用 TGI（Text Generation Inference）部署模型

TGI 是 Hugging Face 推出的推理工具包，可简化大模型部署并针对文本生成进行加速。它通过张量并行、GPU 分布式计算、以及专用 CUDA 内核实现高吞吐。同时内置生产级功能：分布式链路追踪、Prometheus 指标、水印安全机制等。详见[官方文档](https://huggingface.co/docs/text-generation-inference/en/index)。

你可以任选以下方式部署：

1. **本地 Docker 部署**（需本机 GPU & Docker）：取消下方代码注释即可。  
2. **Google Cloud GKE 部署**：参考官方教程[《在 GKE 上使用 GPU 与 Hugging Face TGI 提供 Gemma 模型服务》](https://cloud.google.com/kubernetes-engine/docs/tutorials/serve-gemma-gpu-tgi)。

两种方案都会暴露一个 HTTP 端点，供你发送请求并实时获取生成结果。